In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pmdarima as pm
import itertools
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.graphics.tsaplots  import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.api import Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from arch import arch_model
from prophet import Prophet

In [2]:
exchange = pd.read_csv('./data/exchange.csv', index_col=0)

In [3]:
exchange['date'] = pd.to_datetime(exchange['date'])

In [4]:
forecasting_horizon = int(exchange.shape[0]*0.2) 

In [5]:
index = ['0', '1', '2', '3', '4', '5', '6', 'OT']
columns = ['Actual', 'ARIMA', 'ARIMA 2',\
           'Simple Exponential Smoothing','Holt\'s Exponential Smoothing', \
          'Prophet', 'Prophet+ARIMA']


results = {}
results = pd.DataFrame(results, index=index, columns =columns)
results = results.drop(['Actual'], axis='columns')

In [6]:
exchange0 = exchange[['date','0']]
exchange1 = exchange[['date','1']]
exchange2 = exchange[['date','2']]
exchange3 = exchange[['date','3']]
exchange4 = exchange[['date','4']]
exchange5 = exchange[['date','5']]
exchange6 = exchange[['date','6']]
exchangeOT = exchange[['date','OT']]

In [7]:
exchange0.rename(columns={'date':'ds', '0':'y'}, inplace =True)
exchange1.rename(columns={'date':'ds', '1':'y'}, inplace =True)
exchange2.rename(columns={'date':'ds', '2':'y'}, inplace =True)
exchange3.rename(columns={'date':'ds', '3':'y'}, inplace =True)
exchange4.rename(columns={'date':'ds', '4':'y'}, inplace =True)
exchange5.rename(columns={'date':'ds', '5':'y'}, inplace =True)
exchange6.rename(columns={'date':'ds', '6':'y'}, inplace =True)
exchangeOT.rename(columns={'date':'ds', 'OT':'y'}, inplace =True)

C:\Users\Seungyun\AppData\Local\Temp\ipykernel_13420\1671268679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange0.rename(columns={'date':'ds', '0':'y'}, inplace =True)
C:\Users\Seungyun\AppData\Local\Temp\ipykernel_13420\1671268679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange1.rename(columns={'date':'ds', '1':'y'}, inplace =True)
C:\Users\Seungyun\AppData\Local\Temp\ipykernel_13420\1671268679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

# 0

In [8]:
data = exchange0

In [9]:
#ARIMA(1,1,0)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[0] = arima
#results['ARIMA-GARCH'].iloc[0] = arch

In [10]:
#ARIMA(1,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[0] = arima
#results['ARIMA-GARCH 2'].iloc[0] = arch

C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: Convergence

C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Seungyun\anaconda3\lib\site-packages\statsmodels\base\model.py:604: Convergence

In [11]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[0] = ses
results['Holt\'s Exponential Smoothing'].iloc[0] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[0] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[0] = prophet
results['Prophet+ARIMA'].iloc[0] = prophet_arima

15:38:48 - cmdstanpy - INFO - Chain [1] start processing
15:38:51 - cmdstanpy - INFO - Chain [1] done processing
15:39:10 - cmdstanpy - INFO - Chain [1] start processing
15:39:14 - cmdstanpy - INFO - Chain [1] done processing
15:39:30 - cmdstanpy - INFO - Chain [1] start processing
15:39:32 - cmdstanpy - INFO - Chain [1] done processing
15:39:51 - cmdstanpy - INFO - Chain [1] start processing
15:39:53 - cmdstanpy - INFO - Chain [1] done processing
15:40:09 - cmdstanpy - INFO - Chain [1] start processing
15:40:12 - cmdstanpy - INFO - Chain [1] done processing
15:40:27 - cmdstanpy - INFO - Chain [1] start processing
15:40:29 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:48 - cmdstanpy - INFO - Chain [1] done processing
15:41:05 - cmdstanpy - INFO - Chain [1] start processing
15:41:08 - cmdstanpy - INFO - Chain [1] done processing
15:41:26 - cmdstanpy - INFO - Chain [1] start processing
15:41:29 - cmdstanpy - INFO - Chain [1]

16:01:06 - cmdstanpy - INFO - Chain [1] done processing
16:01:23 - cmdstanpy - INFO - Chain [1] start processing
16:01:25 - cmdstanpy - INFO - Chain [1] done processing
16:01:43 - cmdstanpy - INFO - Chain [1] start processing
16:01:45 - cmdstanpy - INFO - Chain [1] done processing
16:02:01 - cmdstanpy - INFO - Chain [1] start processing
16:02:03 - cmdstanpy - INFO - Chain [1] done processing
16:02:21 - cmdstanpy - INFO - Chain [1] start processing
16:02:23 - cmdstanpy - INFO - Chain [1] done processing
16:02:44 - cmdstanpy - INFO - Chain [1] start processing
16:02:46 - cmdstanpy - INFO - Chain [1] done processing
16:03:03 - cmdstanpy - INFO - Chain [1] start processing
16:03:05 - cmdstanpy - INFO - Chain [1] done processing
16:03:23 - cmdstanpy - INFO - Chain [1] start processing
16:03:25 - cmdstanpy - INFO - Chain [1] done processing
16:03:46 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] done processing
16:04:11 - cmdstanpy - INFO - Chain [1] 

16:24:59 - cmdstanpy - INFO - Chain [1] done processing
16:25:16 - cmdstanpy - INFO - Chain [1] start processing
16:25:19 - cmdstanpy - INFO - Chain [1] done processing
16:25:34 - cmdstanpy - INFO - Chain [1] start processing
16:25:38 - cmdstanpy - INFO - Chain [1] done processing
16:25:55 - cmdstanpy - INFO - Chain [1] start processing
16:25:58 - cmdstanpy - INFO - Chain [1] done processing
16:26:14 - cmdstanpy - INFO - Chain [1] start processing
16:26:17 - cmdstanpy - INFO - Chain [1] done processing
16:26:33 - cmdstanpy - INFO - Chain [1] start processing
16:26:36 - cmdstanpy - INFO - Chain [1] done processing
16:26:51 - cmdstanpy - INFO - Chain [1] start processing
16:26:54 - cmdstanpy - INFO - Chain [1] done processing
16:27:12 - cmdstanpy - INFO - Chain [1] start processing
16:27:14 - cmdstanpy - INFO - Chain [1] done processing
16:27:29 - cmdstanpy - INFO - Chain [1] start processing
16:27:32 - cmdstanpy - INFO - Chain [1] done processing
16:27:51 - cmdstanpy - INFO - Chain [1] 

16:48:01 - cmdstanpy - INFO - Chain [1] done processing
16:48:17 - cmdstanpy - INFO - Chain [1] start processing
16:48:20 - cmdstanpy - INFO - Chain [1] done processing
16:48:34 - cmdstanpy - INFO - Chain [1] start processing
16:48:38 - cmdstanpy - INFO - Chain [1] done processing
16:48:52 - cmdstanpy - INFO - Chain [1] start processing
16:48:55 - cmdstanpy - INFO - Chain [1] done processing
16:49:09 - cmdstanpy - INFO - Chain [1] start processing
16:49:12 - cmdstanpy - INFO - Chain [1] done processing
16:49:26 - cmdstanpy - INFO - Chain [1] start processing
16:49:30 - cmdstanpy - INFO - Chain [1] done processing
16:49:44 - cmdstanpy - INFO - Chain [1] start processing
16:49:48 - cmdstanpy - INFO - Chain [1] done processing
16:50:00 - cmdstanpy - INFO - Chain [1] start processing
16:50:04 - cmdstanpy - INFO - Chain [1] done processing
16:50:17 - cmdstanpy - INFO - Chain [1] start processing
16:50:21 - cmdstanpy - INFO - Chain [1] done processing
16:50:33 - cmdstanpy - INFO - Chain [1] 

17:09:38 - cmdstanpy - INFO - Chain [1] done processing
17:09:57 - cmdstanpy - INFO - Chain [1] start processing
17:10:00 - cmdstanpy - INFO - Chain [1] done processing
17:10:17 - cmdstanpy - INFO - Chain [1] start processing
17:10:20 - cmdstanpy - INFO - Chain [1] done processing
17:10:35 - cmdstanpy - INFO - Chain [1] start processing
17:10:38 - cmdstanpy - INFO - Chain [1] done processing
17:10:53 - cmdstanpy - INFO - Chain [1] start processing
17:10:56 - cmdstanpy - INFO - Chain [1] done processing
17:11:11 - cmdstanpy - INFO - Chain [1] start processing
17:11:14 - cmdstanpy - INFO - Chain [1] done processing
17:11:29 - cmdstanpy - INFO - Chain [1] start processing
17:11:32 - cmdstanpy - INFO - Chain [1] done processing
17:11:47 - cmdstanpy - INFO - Chain [1] start processing
17:11:50 - cmdstanpy - INFO - Chain [1] done processing
17:12:09 - cmdstanpy - INFO - Chain [1] start processing
17:12:12 - cmdstanpy - INFO - Chain [1] done processing
17:12:27 - cmdstanpy - INFO - Chain [1] 

17:33:56 - cmdstanpy - INFO - Chain [1] done processing
17:34:14 - cmdstanpy - INFO - Chain [1] start processing
17:34:17 - cmdstanpy - INFO - Chain [1] done processing
17:34:32 - cmdstanpy - INFO - Chain [1] start processing
17:34:35 - cmdstanpy - INFO - Chain [1] done processing
17:34:53 - cmdstanpy - INFO - Chain [1] start processing
17:34:56 - cmdstanpy - INFO - Chain [1] done processing
17:35:16 - cmdstanpy - INFO - Chain [1] start processing
17:35:19 - cmdstanpy - INFO - Chain [1] done processing
17:35:35 - cmdstanpy - INFO - Chain [1] start processing
17:35:38 - cmdstanpy - INFO - Chain [1] done processing
17:35:54 - cmdstanpy - INFO - Chain [1] start processing
17:35:56 - cmdstanpy - INFO - Chain [1] done processing
17:36:12 - cmdstanpy - INFO - Chain [1] start processing
17:36:14 - cmdstanpy - INFO - Chain [1] done processing
17:36:30 - cmdstanpy - INFO - Chain [1] start processing
17:36:34 - cmdstanpy - INFO - Chain [1] done processing
17:36:49 - cmdstanpy - INFO - Chain [1] 

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange0_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 1

In [ ]:
data = exchange1

In [ ]:
#ARIMA(1,1,0)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[1] = arima
#results['ARIMA-GARCH'].iloc[1] = arch

In [ ]:
#ARIMA(0,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[1] = arima
#results['ARIMA-GARCH 2'].iloc[1] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[1] = ses
results['Holt\'s Exponential Smoothing'].iloc[1] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[1] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[1] = prophet
results['Prophet+ARIMA'].iloc[1] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange1_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 2

In [ ]:
data = exchange2

In [ ]:
#ARIMA(1,1,0)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[2] = arima
#results['ARIMA-GARCH'].iloc[2] = arch

In [ ]:
#ARIMA(2,1,2)
arima_errors = []
arima_preds1 = []
# #ARIMA(2,1,2)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (2,1,2)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[2] = arima
#results['ARIMA-GARCH 2'].iloc[2] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[2] = ses
results['Holt\'s Exponential Smoothing'].iloc[2] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[2] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[2] = prophet
results['Prophet+ARIMA'].iloc[2] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange2_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 3

In [ ]:
data = exchange3

In [ ]:
#ARIMA(3,1,0)
arima_errors = []
arima_preds1 = []
# #ARIMA(3,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (3,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[3] = arima
#results['ARIMA-GARCH'].iloc[3] = arch

In [ ]:
#ARIMA(0,1,3)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,3)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,3)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[3] = arima
#results['ARIMA-GARCH 2'].iloc[3] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[3] = ses
results['Holt\'s Exponential Smoothing'].iloc[3] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[3] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[3] = prophet
results['Prophet+ARIMA'].iloc[3] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange3_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 4

In [ ]:
data = exchange4

In [ ]:
#ARIMA(0,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[4] = arima
#results['ARIMA-GARCH'].iloc[4] = arch

In [ ]:
#ARIMA(1,2,5)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,2,5)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,2,5)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[4] = arima
#results['ARIMA-GARCH 2'].iloc[4] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[4] = ses
results['Holt\'s Exponential Smoothing'].iloc[4] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[4] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[4] = prophet
results['Prophet+ARIMA'].iloc[4] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange4_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 5

In [ ]:
data = exchange5

In [ ]:
#ARIMA(0,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
arch_abs_errors = np.array(arch_errors)
arch = arch_abs_errors.mean()

results['ARIMA'].iloc[5] = arima
#results['ARIMA-GARCH'].iloc[5] = arch

In [ ]:
#ARIMA(0,1,3)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,3)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,3)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[5] = arima
#results['ARIMA-GARCH 2'].iloc[5] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[5] = ses
results['Holt\'s Exponential Smoothing'].iloc[5] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[5] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[5] = prophet
results['Prophet+ARIMA'].iloc[5] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange5_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# 6

In [ ]:
data = exchange6

In [ ]:
#ARIMA(0,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[6] = arima
#results['ARIMA-GARCH'].iloc[6] = arch

In [ ]:
#ARIMA(1,1,0)
arima_errors = []
# arima_preds1 = []
# #ARIMA(1,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[6] = arima
#results['ARIMA-GARCH 2'].iloc[6] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
#Holt-Winter
hw_errors = []
hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[6] = ses
results['Holt\'s Exponential Smoothing'].iloc[6] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[6] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[6] = prophet
results['Prophet+ARIMA'].iloc[6] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchange6_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')

# OT

In [ ]:
data = exchangeOT

In [ ]:
#ARIMA(0,1,1)
arima_errors = []
arima_preds1 = []
# #ARIMA(0,1,1)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (0,1,1)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA'].iloc[7] = arima
#results['ARIMA-GARCH'].iloc[7] = arch

In [ ]:
#ARIMA(1,1,0)
arima_errors = []
arima_preds1 = []
# #ARIMA(1,1,0)-GARCH(1,1)
# arch_errors = []
# arch_preds1 = []
order = (1,1,0)


for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    arima_model = ARIMA(data['y'][:target-4], order = order)
    arima_model_fit = arima_model.fit()
    arima_pred = arima_model_fit.forecast(steps=5).iloc[-1].item()
    arima_error = arima_pred - data['y'][target]
    arima_abs_error = abs(arima_error)
    arima_preds1.append(arima_pred)
    arima_errors.append(arima_abs_error)
    
#     arima_residuals = arima_model_fit.resid
#     garch = arch_model(arima_residuals, p=1,q=1)
#     garch_fitted = garch.fit()
#     predicted_mu = arima_model_fit.predict(n_periods=5).iloc[-1]
#     garch_forecast = garch_fitted.forecast(horizon=5)
#     predicted_et = garch_forecast.mean['h.1'].iloc[-1]
#     arch_pred = predicted_mu + predicted_et
#     arch_error = arch_pred - data['y'][target]
#     arch_abs_error = abs(arch_error)
#     arch_preds1.append(arch_pred)
#     arch_errors.append(arch_abs_error)

    
arima_abs_errors = np.array(arima_errors)
arima = arima_abs_errors.mean()
# arch_abs_errors = np.array(arch_errors)
# arch = arch_abs_errors.mean()

results['ARIMA 2'].iloc[7] = arima
#results['ARIMA-GARCH 2'].iloc[7] = arch

In [ ]:
#SES
ses_errors = []
ses_preds1 = []
#Holt
holt_errors = []
holt_preds1 = []
# #Holt-Winter
# hw_errors = []
# hw_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    ses_model = SimpleExpSmoothing(data['y'][:target-4])
    ses_model_fit = ses_model.fit()
    ses_pred = ses_model_fit.forecast(5).iloc[-1].item()
    ses_error = ses_pred - data['y'][target]
    ses_abs_error = abs(ses_error)
    ses_preds1.append(ses_pred)
    ses_errors.append(ses_abs_error)
    
    holt_model = Holt(data['y'][:target-4])
    holt_model_fit = holt_model.fit()
    holt_pred = holt_model_fit.forecast(5).iloc[-1].item()
    holt_error = holt_pred - data['y'][target]
    holt_abs_error = abs(holt_error)
    holt_preds1.append(holt_pred)
    holt_errors.append(holt_abs_error)
    
#     hw_model = ExponentialSmoothing(data['y'][:-target-4], seasonal_periods=365, trend='add', seasonal='add')
#     hw_model_fit = hw_model.fit()
#     hw_pred = hw_model_fit.forecast(5).iloc[-1].item()
#     hw_error = hw_pred - data['y'][target]
#     hw_abs_error = abs(hw_error)
#     hw_preds1.append(hw_pred)
#     hw_errors.append(hw_abs_error)
    
ses_abs_errors = np.array(ses_errors)
ses = ses_abs_errors.mean()
holt_abs_errors = np.array(holt_errors)
holt = holt_abs_errors.mean()
# hw_abs_errors = np.array(hw_errors)
# hw = hw_abs_errors.mean()

results['Simple Exponential Smoothing'].iloc[7] = ses
results['Holt\'s Exponential Smoothing'].iloc[7] = holt
#results['Holt-Winter\'s Exponential Smoothing'].iloc[7] = hw

In [ ]:
#prophet
prophet_errors = []
prophet_preds1 = []
#prophet-ARIMA
prophet_arima_errors = []
prophet_arima_preds1 = []

for i in range(forecasting_horizon) :
    target = data.shape[0]-forecasting_horizon+i
    date = data['ds'].iloc[target] - data['ds'].iloc[target-5]
    m = Prophet()
    m.fit(data.iloc[:target-4])
    future = m.make_future_dataframe(periods=date.days)
    forecast = m.predict(future)
    prophet_pred = forecast['yhat'].iloc[-1]
    prophet_error = prophet_pred - data['y'][target]
    prophet_abs_error = abs(prophet_error)
    prophet_preds1.append(prophet_pred)
    prophet_errors.append(prophet_abs_error)
    
    
    error_series = data['y'][:target-4] - forecast['yhat'].iloc[:-date.days]
    prophet_arima_model = pm.auto_arima(error_series, start_p = 0, start_q = 0, max_p = 5, max_q = 5, \
                                        seasonal = False, information_criteria = 'bic')
    prophet_arima_pred = prophet_arima_model.predict(n_periods=5).iloc[-1].item() + prophet_pred
    prophet_arima_error = prophet_arima_pred - data['y'][target]
    prophet_arima_abs_error = abs(prophet_arima_error)
    prophet_arima_preds1.append(prophet_arima_pred)
    prophet_arima_errors.append(prophet_arima_abs_error)    

    
prophet_abs_errors = np.array(prophet_errors)
prophet = prophet_abs_errors.mean()
prophet_arima_abs_errors = np.array(prophet_arima_errors)
prophet_arima = prophet_arima_abs_errors.mean()

results['Prophet'].iloc[7] = prophet
results['Prophet+ARIMA'].iloc[7] = prophet_arima

In [ ]:
data_pred =pd.DataFrame(index= data['ds'][-forecasting_horizon:], columns=columns)
data_pred['Actual'] = list(data['y'][data.shape[0]-forecasting_horizon:])
data_pred['ARIMA'] = arima_preds1
#data_pred['ARIMA-GARCH'] = arch_preds1
data_pred['ARIMA 2'] = arima_preds2_1
#data_pred['ARIMA-GARCH 2'] = arch_preds2_1
data_pred['Simple Exponential Smoothing'] = ses_preds1
data_pred['Holt\'s Exponential Smoothing'] = holt_preds1
#data_pred['Holt-Winter\'s Exponential Smoothing'] = hw_preds1
data_pred['Prophet'] = prophet_preds1
data_pred['Prophet+ARIMA'] = prophet_arima_preds1
data_pred.to_csv('./results/exchangeOT_5step_forecasting.csv')

In [ ]:
results.to_csv('./results/exchange_5step_rate_mae.csv')